In [17]:
import requests
from datetime import datetime
import pandas as pd
import re 
import csv
import json
import time
from datetime import date

In [ ]:
now = datetime.today()

In [18]:
'''
- Url "https://www.aeonthailand.co.th/th/api/v1/stores"
- Respone json type dict
- keys after get ['stores', 'time']
- after unpack dict with 'stores' data will be list type for each store
- example  json_data['stores'][0] 
- output
    {'id': 90, 'tel': '02-136-1364', 'fax': None, 'latitude': 13.759365, 'longitude': 100.617455, 'store_type_id': 2, 
    'name': 'รามคำแหง 49', 'address': '361 ถนนรามคำแหง แขวงหัวหมาก เขตบางกะปิ กรุงเทพมหานคร 10240'}
- each list contain dict 
- keys ['id', 'tel', 'fax', 'latitude', 'longitude', 'store_type_id', 'name', 'address']
- example  json_data['stores'][0]['name']
- output
    รามคำแหง 49
'''

'\n- Url "https://www.aeonthailand.co.th/th/api/v1/stores"\n- Respone json type dict\n- keys after get [\'stores\', \'time\']\n- after unpack dict with \'stores\' data will be list type for each store\n- example  json_data[\'stores\'][0] \n- output\n    {\'id\': 90, \'tel\': \'02-136-1364\', \'fax\': None, \'latitude\': 13.759365, \'longitude\': 100.617455, \'store_type_id\': 2, \n    \'name\': \'รามคำแหง 49\', \'address\': \'361 ถนนรามคำแหง แขวงหัวหมาก เขตบางกะปิ กรุงเทพมหานคร 10240\'}\n- each list contain dict \n- keys [\'id\', \'tel\', \'fax\', \'latitude\', \'longitude\', \'store_type_id\', \'name\', \'address\']\n- example  json_data[\'stores\'][0][\'name\']\n- output\n    รามคำแหง 49\n'

In [19]:
#delay ช่วงเวลา 1.00 - 1.30 นาที
def LongdoAPIinfo(lat,lng):
    url = "https://api.longdo.com/map/services/address?"
    params = {'lat': lat , 'lon' : lng , 'key' : '1917e02b4ab264ba547fecdd6e82b4aa'}
    response =requests.get(url, params=params)
    payload = response.json()
    province = payload['province']
    return province

In [20]:
word_spilt_general = ['เลขที่',"[0-9]","/","-",'ถนน','หมู่ที่','หมู่','แขวง','เขต','ตำบล','อำเภอ','จังหวัด']
word_spilt_specialcase = ['เลขที่',"[0-9]","/","-",'ถนน','หมู่ที่','หมู่','แขวง','เขต','ตำบล','อำเภอ','จังหวัด']
#word_spilt_bangkok = ['เลขที่',"[0-9]","/","-",'ถนน','หมู่ที่','หมู่','แขวง','เขต','จังหวัด']
remove_flag_list = []

def getAddressDetail(address,lat,lng):
    #print(address)
    #list_address_temp.clear()
    str_address = address.replace("  "," ")
    #print(str_address)

    # remove newline
    if("\n" in str_address) :
        str_address = re.sub('\r','',str_address)
        str_address = re.sub('\n',' ',str_address)
    
    if(re.findall('กรุงเทพมหาคร',str_address)):
        #print(str_address)
        str_address = re.sub(r'กรุงเทพมหาคร',"จังหวัดกรุงเทพมหานคร",str_address)
        
    # fix wrong bangkok
    elif("กรุงเทพมหานคร" in str_address) :
            str_address = re.sub(r'กรุงเทพมหานคร',"จังหวัดกรุงเทพมหานคร",str_address)
    
    if(re.findall(r'\ฯ',str_address)):
        #print(str_address)
        str_address = re.sub(r'\ฯ',"",str_address)
    
    temp = str_address.split()
    
    
    if(temp[-1] == "กทม.") :
        temp[-1] = "จังหวัดกรุงเทพมหานคร"
    elif(temp[-2] == 'กทม.') :
        temp[-2] = "จังหวัดกรุงเทพมหานคร"
    

    if(temp[-1] == 'กรุงเทพ') :
        temp[-1] = "จังหวัดกรุงเทพมหานคร"
    elif(temp[-2] == 'กรุงเทพ') :
        temp[-2] = "จังหวัดกรุงเทพมหานคร"

    if(temp[-1] == 'ชลบุรี') :
        temp[-1] = "จังหวัดชลบุรี"
    elif(temp[-2] == 'ชลบุรี') :
        temp[-2] = "จังหวัดชลบุรี"
    

    temp_word = ""
    for i in temp :
        temp_word = temp_word + " " + i
    
    str_address = temp_word
    
    if(not re.search('จังหวัด',str_address)):
        temp = str_address.split()
        province = LongdoAPIinfo(lat,lng)
        temp[-1] = "จังหวัด" + province
        for i in temp :
            temp_word = temp_word + " " + i
    
        str_address = temp_word
    
    #print(str_address)
    
    
    if(not re.search('จังหวัด',str_address)):
        province = LongdoAPIinfo(lat,lng)
        temp[-1] = province
    
    if(re.findall(r'\ต+[.]',str_address) or re.findall(r'\อ+[.]',str_address) or re.findall(r'\จ+[.]',str_address) ) :
        #print("wrong keyword!")
        flag_tum = False
        flag_aumper = False
        flag_province = False
        if(re.findall(r"\ต.",str_address)) :
            flag_tum  = True

        if(re.findall(r"\อ.",str_address)) :
            flag_aumper = True

        if(re.findall(r"\จ.",str_address)) :
            flag_province = True
        
        #print(flag_tum)
        #print(flag_aumper)
        #print(flag_province)
        str_address = str_address.split(" ")
        
        if flag_tum :
            str_address[-4] = str_address[-4].replace("ต.", "ตำบล")
            #print(str_address[-4])
        if flag_aumper :
            str_address[-3] = str_address[-3].replace("อ.", "อำเภอ")
            #print(str_address[-3])
        if flag_province :
            str_address[-2] = str_address[-2].replace("จ.", "จังหวัด")
            #print(str_address[-2])
        
        temp = ""
        for i in str_address :
            temp = temp + i + " "
        str_address = temp
        
    #print(str_address)
    
    if not (re.findall(r"\จังหวัด",str_address)) :
        #print("test")
        str_address = str_address.split(" ")
        #print(str_address)
        str_address[-2] = "จังหวัด"+str_address[-2]
        temp = ""
        for i in str_address :
            temp = temp + i + " "
        str_address = temp

    str_address = str_address.replace(' ','')
   
    for i in range (len(word_spilt_general)):
        #print(word_spilt_general[i])
        if re.search(word_spilt_general[i],str_address) :
            #print("found!")
            str_address = re.sub(word_spilt_general[i]," ",str_address)
            str_address = str_address.lstrip()
            remove_flag_list.append(True)
        else :
            
            #print("not found!")
            remove_flag_list.append(False)
    
    str_address = str_address.rstrip() 
    
    str_address = str_address.replace("  "," ")
    str_address = str_address.split(" ")

    #print(str_address)
    
    subdistrict = str_address[-3]
    district = str_address[-2]
    province = str_address[-1]
    
    if(subdistrict == "ประชาธิปัติ"):
        subdistrict = "ประชาธิปัตย์"

    #print("แขวง,ตำบล: "+ subdistrict)
    #print("เขต,อำเภอ: "+ district) 
    #print("จังหวัด: "+ province) 
    #print("-----------------------------")
    
    remove_flag_list.clear()
    
    return subdistrict,district,province

In [21]:
def getStorename(id):
    store_type = ""
    if(id == 1):
        store_type = "Maxvalu"
    elif(id == 2):
        store_type = "Maxvalu tanjai"
    return store_type

In [22]:
def getStoretype(id):
    store_type = ""
    if(id == 1):
        store_type = "Supermarket"
    elif(id == 2):
        store_type = "Convenience Stores"
    return store_type

In [23]:
#- keys ['id', 'tel', 'fax', 'latitude', 'longitude', 'store_type_id', 'name', 'address']

In [24]:
  # while i < len(payload['stores']):

In [25]:
def getdataFromAPI():
    url = "https://www.aeonthailand.co.th/th/api/v1/stores"
    response = requests.get(url)
    payload = response.json()
    with open(f'../dataCleandone/MaxValu/raw/maxValu{str(now.date())}.json', 'w') as f:
        json.dump(payload, f)

In [26]:
getdataFromAPI()

In [27]:
f = open(f'../dataCleandone/MaxValu/raw/maxValu{str(now.date())}.json')
payload = json.load(f)

In [28]:
i = 0
list_maxvalu = []
list_maxvalu_tanjai = []
while i < len(payload['stores']):
    store_id = payload['stores'][i]['id']
    name = payload['stores'][i]['name']
    address = payload['stores'][i]['address']
    #subdistrict,district,province = getAddressDetail(address)
    lat = payload['stores'][i]['latitude']
    lng = payload['stores'][i]['longitude']
    subdistrict,district,province = getAddressDetail(address,lat,lng)
    store_type_id = payload['stores'][i]['store_type_id']
    store_name = getStorename(store_type_id)
    store_type =  getStoretype(store_type_id)
   
    dict_maxvaluDetail = {"name" : name , "store_id" : store_id,
                          "name_store" : store_name , "store_type" : store_type,
                          "address" : address , "lat" : lat , "lng" : lng ,
                          "subdistrict" : subdistrict , "district" : district ,"province" : province 
                         }
    if(store_type_id == 1) :
        list_maxvalu.append(dict_maxvaluDetail)
    elif(store_type_id == 2):
        list_maxvalu_tanjai.append(dict_maxvaluDetail)

    i = i + 1
    
with open(f"../dataCleandone/MaxValu/maxValu{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_maxvalu, outfile)
with open(f"../dataCleandone/MaxValu/maxValutanjai{str(now.date())}.json"  , "w") as outfile:
            json.dump(list_maxvalu_tanjai, outfile)


In [29]:
payload_dict = dict_maxvaluDetail

In [30]:
#create dataframe

payload_dataframe = pd.DataFrame(payload_dict,index=[],columns=['code','name','address','subdistrict','district','province','latitude','longitude','store_type_id','list_store_type_name'])


In [ ]:
df_json =  pd.read_json(f"../dataCleandone/MaxValu/maxValu{str(now.date())}.json")
df_json.to_csv(f"../dataCleandone/MaxValu/maxValu{str(now.date())}.csv")

df_json =  pd.read_json(f"../dataCleandone/MaxValu/maxValutanjai{str(now.date())}.json")
df_json.to_csv(f"../dataCleandone/MaxValu/maxValutanjai{str(now.date())}.csv")

In [31]:
payload_dataframe

Empty DataFrame
Columns: [code, name, address, subdistrict, district, province, latitude, longitude, store_type_id, list_store_type_name]
Index: []